# 카트폴 게임 마스터하기

In [1]:
import gym
from gym import wrappers
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import deque
import numpy as np

## OpenAI Gym을 이용하여 게임환경 구축하기


강화학습 예제들을 보면 항상 게임과 연관되어 있습니다. 원래 우리가 궁극적으로 원하는 목표는 어디서든 적응할 수 있는 인공지능이지만, 너무 복잡한 문제이기도 하고 가상 환경을 설계하기도 어렵기 때문에 일단 게임이라는 환경을 사용해 하는 것입니다.

대부분의 게임은 점수 혹은 목표가 있습니다. 점수가 오르거나 목표에 도달하면 일종의 리워드를 받고 원치 않은 행동을 할때는 마이너스 리워드를 주는 경우도 있습니다. 아까 비유를 들었던 달리기를 배울때의 경우를 예로 들면 총 나아간 길이 혹은 목표 도착지 도착 여부로 리워드를 주고 넘어질때 패널티를 줄 수 있을 것입니다. 

게임중에서도 가장 간단한 카트폴이라는 환경을 구축하여 강화학습을 배울 토대를 마련해보겠습니다.

In [2]:
env = gym.make('CartPole-v1')

### 하이퍼파라미터



In [3]:
# 하이퍼파라미터
EPISODES = 50    # 에피소드 반복 횟수
EPS_START = 0.9  # e-greedy threshold 시작 값
EPS_END = 0.05   # e-greedy threshold 최종 값
EPS_DECAY = 200  # e-greedy threshold decay
GAMMA = 0.8      # 
LR = 0.001       # NN optimizer learning rate
BATCH_SIZE = 64  # Q-learning batch size

## DQN 에이전트

In [4]:
class DQNAgent:
    def __init__(self):
        self.model = nn.Sequential(
            nn.Linear(4, 256),
            nn.ReLU(),
            nn.Linear(256, 2)
        )
        self.memory = deque(maxlen=10000)
        self.optimizer = optim.Adam(self.model.parameters(), LR)
        self.steps_done = 0
    
    def act(self, state):
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * self.steps_done / EPS_DECAY)
        self.steps_done += 1
        if random.random() > eps_threshold:
            return self.model(state).data.max(1)[1].view(1, 1)
        else:
            return torch.LongTensor([[random.randrange(2)]])

    def memorize(self, state, action, reward, next_state):
        self.memory.append((state,
                            action,
                            torch.FloatTensor([reward]),
                            torch.FloatTensor([next_state])))
    
    def learn(self):
        """Experience Replay"""
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        states, actions, rewards, next_states = zip(*batch)

        states = torch.cat(states)
        actions = torch.cat(actions)
        rewards = torch.cat(rewards)
        next_states = torch.cat(next_states)

        current_q = self.model(states).gather(1, actions)
        max_next_q = self.model(next_states).detach().max(1)[0]
        expected_q = rewards + (GAMMA * max_next_q)
        
        loss = F.mse_loss(current_q.squeeze(), expected_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

## 학습 준비하기

드디어 만들어둔 DQNAgent를 인스턴스화 합니다.
그리고 `gym`을 이용하여 `CartPole-v0`환경도 준비합니다.
자, 이제 `agent` 객체를 이용하여 `CartPole-v0` 환경과 상호작용을 통해 게임을 배우도록 하겠습니다.
학습 진행을 기록하기 위해 `score_history` 리스트를 이용하여 점수를 저장하겠습니다.

In [5]:
agent = DQNAgent()
env = gym.make('CartPole-v0')
score_history = []

## 학습 시작

In [6]:
for e in range(1, EPISODES+1):
    state = env.reset()
    steps = 0
    while True:
        env.render()
        state = torch.FloatTensor([state])
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action.item())

        # negative reward when attempt ends
        if done:
            reward = -1

        agent.memorize(state, action, reward, next_state)
        agent.learn()

        state = next_state
        steps += 1

        if done:
            print("에피소드:{0} 점수: {1}".format(e, steps))
            score_history.append(steps)
            break

에피소드:1 점수: 11
에피소드:2 점수: 32
에피소드:3 점수: 10
에피소드:4 점수: 36
에피소드:5 점수: 13
에피소드:6 점수: 17
에피소드:7 점수: 9
에피소드:8 점수: 13
에피소드:9 점수: 11
에피소드:10 점수: 28
에피소드:11 점수: 11
에피소드:12 점수: 12
에피소드:13 점수: 9
에피소드:14 점수: 20
에피소드:15 점수: 11
에피소드:16 점수: 11
에피소드:17 점수: 10
에피소드:18 점수: 15
에피소드:19 점수: 13
에피소드:20 점수: 11
에피소드:21 점수: 13
에피소드:22 점수: 22
에피소드:23 점수: 26
에피소드:24 점수: 59
에피소드:25 점수: 30
에피소드:26 점수: 22
에피소드:27 점수: 26
에피소드:28 점수: 25
에피소드:29 점수: 57
에피소드:30 점수: 83
에피소드:31 점수: 62
에피소드:32 점수: 45
에피소드:33 점수: 62
에피소드:34 점수: 80
에피소드:35 점수: 88
에피소드:36 점수: 57
에피소드:37 점수: 52
에피소드:38 점수: 45
에피소드:39 점수: 49
에피소드:40 점수: 63
에피소드:41 점수: 61
에피소드:42 점수: 75
에피소드:43 점수: 52
에피소드:44 점수: 81
에피소드:45 점수: 98
에피소드:46 점수: 129
에피소드:47 점수: 153
에피소드:48 점수: 169
에피소드:49 점수: 120
에피소드:50 점수: 144
